In [ ]:
import numpy as np # linear algebra
import pandas as pd
import xgboost as xgb
import csv as csv
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import cross_val_score,KFold
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV   #Performing grid search
from scipy.stats import skew
from collections import OrderedDict
from sklearn.inspection import permutation_importance
import shap
from sklearn.preprocessing import MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold, KFold
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
from sklearn.metrics import make_scorer
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform
import joblib
import numbers
import torch
from torch_geometric.loader import DataLoader  

In [ ]:
plt.style.use('style.mplstyle')

In [ ]:
def set_seed(seed):
    import random
    random.seed(seed) 
    np.random.seed(seed) 
    torch.manual_seed(seed) 
    torch.cuda.manual_seed(seed)  
    torch.backends.cudnn.deterministic = True  
    torch.backends.cudnn.benchmark = False 

set_seed(42) 

In [ ]:
def save_model_with_joblib(model, filename):
    joblib.dump(model, filename)

In [ ]:
import torch

def load_data_list(file_path):
    return torch.load(file_path)

############## Tm ##############
Tm_load_file_path = '../dataset/Tm_data_list.pt' 
Tm_load_train_dataset_path = '../dataset/train_Tm_data_list.pt' 
Tm_load_val_dataset_path = '../dataset/val_Tm_data_list.pt' 
Tm_load_test_dataset_path = '../dataset/test_Tm_data_list.pt' 


Tm_loaded_data_list = load_data_list(Tm_load_file_path)
Tm_loaded_train_data_list = load_data_list(Tm_load_train_dataset_path)
Tm_loaded_val_data_list = load_data_list(Tm_load_val_dataset_path)
Tm_loaded_test_data_list = load_data_list(Tm_load_test_dataset_path)

Tm_train_loader = DataLoader(Tm_loaded_train_data_list, batch_size=32, shuffle=True)  
Tm_val_loader = DataLoader(Tm_loaded_val_data_list, batch_size=32, shuffle=False)  
Tm_test_loader = DataLoader(Tm_loaded_test_data_list, batch_size=32, shuffle=False) 


############## conductivity ##############
conductivity_load_file_path = 'conductivity_data_list.pt'
conductivity_load_train_dataset_path = 'train_conductivity_data_list.pt' 
conductivity_load_val_dataset_path = 'val_conductivity_data_list.pt'
conductivity_load_test_dataset_path = 'test_conductivity_data_list.pt' 

conductivity_loaded_data_list = load_data_list(conductivity_load_file_path)
conductivity_loaded_train_data_list = load_data_list(conductivity_load_train_dataset_path)
conductivity_loaded_val_data_list = load_data_list(conductivity_load_val_dataset_path)
conductivity_loaded_test_data_list = load_data_list(conductivity_load_test_dataset_path)


conductivity_train_loader = DataLoader(conductivity_loaded_train_data_list, batch_size=32, shuffle=True)  
conductivity_val_loader = DataLoader(conductivity_loaded_val_data_list, batch_size=32, shuffle=False)  
conductivity_test_loader = DataLoader(conductivity_loaded_test_data_list, batch_size=32, shuffle=False)  

######################ECW######################
IL_ECW_save_total_dataset_path = '../dataset/IL_ECW_data_list.pt' 
IL_ECW_save_train_dataset_path = '../dataset/train_IL_ECW_data_list.pt' 
IL_ECW_save_val_dataset_path = '../dataset/val_IL_ECW_data_list.pt'
IL_ECW_save_test_dataset_path = '../dataset/test_IL_ECW_data_list.pt'

IL_ECW_loaded_data_list = load_data_list(IL_ECW_save_total_dataset_path)
IL_ECW_loaded_train_data_list = load_data_list(IL_ECW_save_train_dataset_path)
IL_ECW_loaded_val_data_list = load_data_list(IL_ECW_save_val_dataset_path)
IL_ECW_loaded_test_data_list = load_data_list(IL_ECW_save_test_dataset_path)

IL_ECW_train_loader = DataLoader(IL_ECW_loaded_train_data_list, batch_size=32, shuffle=True)  
IL_ECW_val_loader = DataLoader(IL_ECW_loaded_val_data_list, batch_size=32, shuffle=False)  
IL_ECW_test_loader = DataLoader(IL_ECW_loaded_test_data_list, batch_size=32, shuffle=False)  

In [ ]:
import xgboost as xgb
import optuna
import time
import torch
import numpy as np
from sklearn.metrics import mean_squared_error

def extract_features_targets(data_list, scale=1, feature = "fp"):
    X = []
    Y = []
    for data in data_list:
        if feature == "2Ddescriptors":
            moldescriptor = data.moldescriptor.numpy().flatten()
            X.append(moldescriptor)
            
        elif feature == "fp":
            fp = data.morgan_fp.numpy().flatten()
            X.append(fp)
            
        target = data.y.numpy().flatten()
        Y.append(scale*target)

    
    X = np.array(X)
    Y = np.array(Y)
    return X, Y

def XGBoost_model_optuna(train_list, val_list, scale=1, feature = "2Ddescriptors"):

    start_time = time.time()
    
    X_train, y_train = extract_features_targets(train_list, scale, feature)
    
    X_val, y_val = extract_features_targets(val_list, scale, feature)
    
    def objective(trial):
        param = {
            "max_depth": trial.suggest_int("max_depth", 3, 10),  
            "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2, log=True), 
            "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=50), 
            "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),  
            "gamma": trial.suggest_float("gamma", 0, 5), 
            "subsample": trial.suggest_float("subsample", 0.6, 1.0), 
            "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0), 
            "reg_alpha": trial.suggest_float("reg_alpha", 1e-8, 10, log=True),  
            "reg_lambda": trial.suggest_float("reg_lambda", 1e-8, 10, log=True),  
        }

        model = xgb.XGBRegressor(
            objective="reg:squarederror",
            **param,
            random_state=42,
            n_jobs=-1
        )

        model.fit(
            X_train, y_train,
            eval_set=[(X_val, y_val)],
            early_stopping_rounds=30,
            verbose=False
        )

        y_pred = model.predict(X_val)

        return mean_squared_error(y_val, y_pred)

    study = optuna.create_study(direction="minimize")
    study.optimize(objective, n_trials=100, timeout=600) 

    best_params = study.best_params
    
    best_model = xgb.XGBRegressor(
        objective="reg:squarederror",
        **best_params,
        random_state=42,
        n_jobs=-1
    )
    best_model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        early_stopping_rounds=30,
        verbose=False
    )
    
    end_time = time.time()
    
    y_final_pred = best_model.predict(X_val)
    final_mse = mean_squared_error(y_val, y_final_pred)
    print("MSE: {:.4f}".format(final_mse))
    
    return best_model

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluate_model(model, val_list, scale=1, figname="model_evaluation",
                   figsize=(2.3, 2.3), output=False, feature = "2Ddescriptors"):
    
    X_test, y_test = extract_features_targets(val_list, scale, feature)
    
    plt.rcParams["font.family"] = "Arial"
    
    y_pred = model.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE (Mean Squared Error): {mse:.2f}")
    
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE (Mean Absolute Error): {mae:.2f}")
    
    rmse = np.sqrt(mse)  
    print(f"RMSE (Root Mean Squared Error): {rmse:.2f}")
    
    r2 = r2_score(y_test, y_pred)
    print(f"R^2 Score: {r2:.2f}")
   
    f, ax = plt.subplots(figsize=figsize) 
    ax.scatter(y_pred, y_test, s=0.5) 

    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    ax.plot([min_val, max_val], [min_val, max_val], color="red", linestyle="--") 
    
    plt.xlabel("Predicted Value") 
    plt.ylabel("Ground Truth")  
    
    text_str = f"MAE = {mae:.2f}\nMSE = {mse:.2f}\nR² = {r2:.2f}"
    plt.text(
        0.05, 0.95, text_str,
        transform=ax.transAxes, 
        fontsize=7, color="black",
        verticalalignment="top", horizontalalignment="left",
        bbox=dict(facecolor='white', alpha=0.5, edgecolor='none')
    )

    for spine in ax.spines.values():
        spine.set_linewidth(1)
    
    if output:
        plt.savefig(f"{figname}.png", dpi=300, bbox_inches="tight")
    
    plt.show()
    return mae, mse, r2

In [ ]:
Tm_xgb_model_fp = XGBoost_model_optuna(
    train_list=Tm_loaded_train_data_list,
    val_list=Tm_loaded_val_data_list,
    scale=1, feature = "fp")

evaluate_model(Tm_xgb_model_fp, Tm_loaded_test_data_list, scale=1, figname="Tm_xgboost_model_evaluation",
                   figsize=(1.75, 1.75), output=True, feature = "fp")

save_model_with_joblib(Tm_xgb_model_fp, '../model/Tm_xgb_model.joblib')

In [ ]:
IL_ECW_xgb_model_fp = XGBoost_model_optuna(
    train_list=IL_ECW_loaded_train_data_list,
    val_list=IL_ECW_loaded_val_data_list,
    target_index=0, feature = "fp")

evaluate_model(IL_ECW_xgb_model_fp, IL_ECW_loaded_test_data_list, target_index=0, figname="IL_ECW_xgboost_model_evaluation",
                   figsize=(1.75, 1.75), output=True, feature = "fp")

save_model_with_joblib(IL_ECW_xgb_model_fp, '../model/IL_ECW_xgb_model.joblib')

In [ ]:
conductivity_xgb_model_fp = XGBoost_model_optuna(
    train_list=conductivity_loaded_train_data_list,
    val_list=conductivity_loaded_val_data_list,
    scale=10, feature = "fp")

evaluate_model(conductivity_xgb_model_fp, conductivity_loaded_test_data_list, scale=10, figname="conductivity_xgboost_model_evaluation",
                   figsize=(2.3, 2.3), output=False, feature = "fp")

save_model_with_joblib(conductivity_xgb_model_fp, 'conductivity_xgb_model.joblib')